In [8]:
import pandas as pd
df = pd.read_csv('./id31.csv')
print("\n Input Data Set is:\n", df)


 Input Data Set is:
   Refund Martial status Taxable Income Cheat
0    Yes         Single             GK    No
1     No        Married             GK    No
2     No         Single             LK    No
3    Yes        Married             GK    No
4     No       Divorced             GK   Yes
5     No        Married             LK    No
6    Yes       Divorced             GK    No
7     No         Single             GK   Yes
8     No        Married             LK    No
9     No         Single             GK   Yes


In [9]:
t = df.keys()[-1]
print('Target Attribute is: ', t)
# Get the attribute names from input dataset
attribute_names = list(df.keys())
#Remove the target attribute from the attribute names list
attribute_names.remove(t) 
print('Predicting Attributes: ', attribute_names)

Target Attribute is:  Cheat
Predicting Attributes:  ['Refund', 'Martial status', 'Taxable Income']


In [10]:
#Function to calculate the entropy of collection S
import math
def entropy(probs): 
    return sum( [-prob*math.log(prob, 2) for prob in probs])

#Function to calulate the entropy of the given Data Sets/List with 
#respect to target attributes
def entropy_of_list(ls,value):  
    from collections import Counter
    cnt = Counter(x for x in ls)# Counter calculates the propotion of class
    print('Target attribute class count(Yes/No)=',dict(cnt))
    total_instances = len(ls)  
    print("Total no of instances/records associated with {0} is: {1}".format(value,total_instances ))
    probs = [x / total_instances for x in cnt.values()]  # x means no of YES/NO
    return entropy(probs) # Call Entropy

In [11]:
def information_gain(df, split_attribute, target_attribute,battr):
    print("\n\n-----Information Gain Calculation of ",split_attribute, " --------") 
    df_split = df.groupby(split_attribute) # group the data based on attribute values
    glist=[]
    for gname,group in df_split:
        print('Grouped Attribute Values \n',group)
        glist.append(gname) 
    
    nobs = len(df.index)    
    df_agg1=df_split.agg({target_attribute:lambda x:entropy_of_list(x, glist.pop())})
    df_agg2=df_split.agg({target_attribute :lambda x:len(x)/nobs})
    
    df_agg1.columns=['Entropy']
    df_agg2.columns=['Proportion']
    
    # Calculate Information Gain:
    new_entropy = sum( df_agg1['Entropy'] * df_agg2['Proportion'])
    if battr !='S':
        old_entropy = entropy_of_list(df[target_attribute],'S-'+df.iloc[0][df.columns.get_loc(battr)])
    else:
        old_entropy = entropy_of_list(df[target_attribute],battr)
    return old_entropy - new_entropy

In [12]:
def id3(df, target_attribute, attribute_names, default_class=None,default_attr='S'):
    
    from collections import Counter
    cnt = Counter(x for x in df[target_attribute])# class of YES /NO
   
    ## First check: Is this split of the dataset homogeneous?
    if len(cnt) == 1:
        return next(iter(cnt))  # next input data set, or raises StopIteration when EOF is hit.
    
    ## Second check: Is this split of the dataset empty? if yes, return a default value
    elif df.empty or (not attribute_names):
        return default_class  # Return None for Empty Data Set
    
    ## Otherwise: This dataset is ready to be divided up!
    else:
        # Get Default Value for next recursive call of this function:
        default_class = max(cnt.keys()) #No of YES and NO Class
        # Compute the Information Gain of the attributes:
        gainz=[]
        for attr in attribute_names:
            ig= information_gain(df, attr, target_attribute,default_attr)
            gainz.append(ig)
            print('Information gain of ',attr,' is : ',ig)
        
        index_of_max = gainz.index(max(gainz))               # Index of Best Attribute
        best_attr = attribute_names[index_of_max]            # Choose Best Attribute to split on
        print("\nAttribute with the maximum gain is: ", best_attr)
        # Create an empty tree, to be populated in a moment
        tree = {best_attr:{}} # Initiate the tree with best attribute as a node 
        remaining_attribute_names =[i for i in attribute_names if i != best_attr]
        
        # Split dataset-On each split, recursively call this algorithm.Populate the empty tree with subtrees, which
        # are the result of the recursive call
        for attr_val, data_subset in df.groupby(best_attr):
            subtree = id3(data_subset,target_attribute, remaining_attribute_names,default_class,best_attr)
            tree[best_attr][attr_val] = subtree
        return tree

In [13]:
from pprint import pprint
tree = id3(df,t,attribute_names)
print("\nThe Resultant Decision Tree is:")
pprint(tree)



-----Information Gain Calculation of  Refund  --------
Grouped Attribute Values 
   Refund Martial status Taxable Income Cheat
1     No        Married             GK    No
2     No         Single             LK    No
4     No       Divorced             GK   Yes
5     No        Married             LK    No
7     No         Single             GK   Yes
8     No        Married             LK    No
9     No         Single             GK   Yes
Grouped Attribute Values 
   Refund Martial status Taxable Income Cheat
0    Yes         Single             GK    No
3    Yes        Married             GK    No
6    Yes       Divorced             GK    No
Target attribute class count(Yes/No)= {'No': 4, 'Yes': 3}
Total no of instances/records associated with Yes is: 7
Target attribute class count(Yes/No)= {'No': 3}
Total no of instances/records associated with No is: 3
Target attribute class count(Yes/No)= {'No': 7, 'Yes': 3}
Total no of instances/records associated with S is: 10
Information gain of